In [34]:
import folium
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import time
from IPython.display import clear_output
import threading
import websocket
import time
from IPython.display import display, clear_output


In [35]:
# global variables
dataframes = []
journey_ids = []
journeys = {}
messages = []

In [36]:
def initialize_map():
    carte = folium.Map(location=[48.8566, 2.3522], zoom_start=11)
    return carte

# Create the initial map
carte = initialize_map()

In [37]:
def creer_carte_folium(journeys, current_indices):
    # Créer une carte centrée sur Paris
    carte = folium.Map(location=[48.8566, 2.3522], zoom_start=11)

    # Dictionnaire pour stocker les éléments de la légende
    legend_html = {}

    for line, gdf_journeys in journeys.items():
        for j, gdf_journey in enumerate(gdf_journeys):
            current_index = current_indices[line][j]

            # Créer une liste de coordonnées pour la ligne complète
            locations = [(lat, lon) for lon, lat in zip(gdf_journey.geometry.x, gdf_journey.geometry.y)]
            folium.PolyLine(locations, weight=2, opacity=0.5, dash_array="5, 5", popup=f"RER {line} Path").add_to(carte)

            # Créer une liste de coordonnées pour la progression actuelle
            current_locations = locations[:current_index + 1]

            # Vérifier si current_locations a au moins deux coordonnées
            if len(current_locations) >= 2:
                folium.PolyLine(current_locations, weight=4, popup=f"RER {line} Progress").add_to(carte)

                # Add a marker for the current train position
                current_position = current_locations[-1]
                folium.CircleMarker(current_position, radius=5, fill=True, popup=f"RER {line}").add_to(carte)

            # Add markers for the stations
            for idx, row in gdf_journey.iterrows():
                station_label = f"{line}{idx + 1}"
                folium.CircleMarker(
                    [row.geometry.y, row.geometry.x],
                    radius=3,
                    fill=True,
                    fill_color='white',
                    popup=f"{station_label}: {row['stop_point_name']}"
                ).add_to(carte)

        # Ajouter un élément à la légende pour la ligne actuelle
        legend_html[line] = f'<span style="display: inline-block; width: 10px; height: 10px; margin-right: 5px;"></span>RER {line} '

    # Créer le HTML de la légende
    legend_html = '<div style="position: fixed; bottom: 10px; left: 50%; transform: translateX(-50%); background-color: white; padding: 5px; border-radius: 5px; z-index: 1000;">' + ''.join(legend_html.values()) + '</div>'

    # Ajouter la légende à la carte
    carte.get_root().html.add_child(folium.Element(legend_html))

    return carte

In [38]:
def add_data_to_map(message: str):
    global carte, journeys, journey_ids, messages, dataframes
    print("Ajout de données à la carte")
    print(f"Message : {message}")
    
    # Ignore le message d'en-tête
    if message.startswith("stop_point_id"):
        print("Message d'en-tête ignoré")
        return
    
    if len(journey_ids) < 6 and message.split(",")[1] not in journey_ids and message not in messages:
        journey_id = message.split(",")[1]
        line = journey_id[0]  # Assuming the first character of journey_id represents the line
        journey_ids.append((line, journey_id))
        messages.append(message)
        columns = ["stop_point_id", "journey_id", "arrival_time", "departure_time", "stop_point_name", "longitude", "latitude"]
        df = pd.DataFrame([message.split(",")], columns=columns)
        
        # Convertir les colonnes longitude et latitude en float
        df['longitude'] = df['longitude'].astype(float)
        df['latitude'] = df['latitude'].astype(float)
        
        dataframes.append(df)

        # Concatenate all DataFrames into a single DataFrame
        train_data = pd.concat(dataframes, ignore_index=True)

        journey = train_data[train_data["journey_id"] == journey_id]
        geometry = [Point(xy) for xy in zip(journey["longitude"], journey["latitude"])]
        gdf_journey = gpd.GeoDataFrame(journey, geometry=geometry)

        # Add the GeoDataFrame to the corresponding list in journeys
        if line in journeys:
            journeys[line].append(gdf_journey)
        else:
            journeys[line] = [gdf_journey]

        # Update the map
        current_indices = {l: [len(gdf) - 1 for gdf in gdf_journeys] for l, gdf_journeys in journeys.items()}
        carte = creer_carte_folium(journeys, current_indices)
        
        # Clear the output and display the updated map
        clear_output(wait=True)
        display(carte)

In [39]:
def on_message(ws, message):
    print("Message reçu")
    print(f"Message reçu : {message}")
    add_data_to_map(message)
    
def on_error(ws, error):
    print(f"Erreur : {error}")


def on_close(ws, close_status_code, close_msg):
    print("Connexion fermée")


def on_open(ws):
    print("Connexion ouverte")


def run_websocket():
    global carte
    print("Starting WebSocket client...")
    ws = websocket.WebSocketApp(
        "ws://localhost:8888",
        on_open=on_open,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close,
    )

    # Initialize and display the map
    carte = initialize_map()
    display(carte)

    # Run the WebSocket client in a thread
    wst = threading.Thread(target=ws.run_forever)
    wst.daemon = True
    wst.start()

    try:
        while True:
            time.sleep(1)  # Sleep to prevent high CPU usage
    except KeyboardInterrupt:
        print("Interruption par l'utilisateur")
        ws.close()
        wst.join()  # Ensure the thread finishes before exiting

In [40]:
run_websocket()

Starting WebSocket client...


Connexion ouverte
Message reçu
Message reçu : stop_point:SNCF:87758003:RapidTransit, vehicle_journey:SNCF:2024-07-08:BABY17:1187:RapidTransit, 165800, 165800, Charles-de-Gaulle Étoile, 2.29502, 48.874201
Ajout de données à la carte
Message : stop_point:SNCF:87758003:RapidTransit, vehicle_journey:SNCF:2024-07-08:BABY17:1187:RapidTransit, 165800, 165800, Charles-de-Gaulle Étoile, 2.29502, 48.874201
Erreur : name 'color' is not defined
Message reçu
Message reçu : stop_point:SNCF:87758011:RapidTransit, vehicle_journey:SNCF:2024-07-08:BABY17:1187:RapidTransit, 170200, 170200, La Grande Arche de la Défense, 2.238212, 48.891718
Ajout de données à la carte
Message : stop_point:SNCF:87758011:RapidTransit, vehicle_journey:SNCF:2024-07-08:BABY17:1187:RapidTransit, 170200, 170200, La Grande Arche de la Défense, 2.238212, 48.891718
Erreur : name 'color' is not defined
Interruption par l'utilisateur
Connexion fermée


In [ ]:
# # Simulation de la progression du trajet
# if journeys:
#     max_length = max(len(gdf_journey) for gdf_journeys in journeys.values() for gdf_journey in gdf_journeys)
#     current_indices = {line: [0] * len(gdf_journeys) for line, gdf_journeys in journeys.items()}

#     for i in range(max_length):
#         for line, gdf_journeys in journeys.items():
#             for j, gdf_journey in enumerate(gdf_journeys):
#                 if current_indices[line][j] < len(gdf_journey) - 1:
#                     current_indices[line][j] += 1
#         clear_output(wait=True)
#         carte = creer_carte_folium(journeys, current_indices)
#         display(carte)
#         time.sleep(1)
# else:
#     carte = folium.Map(location=[48.8566, 2.3522], zoom_start=12)  # Display an empty map centered on Paris
#     display(carte)